## Решение - RDD

Описание данных:  
У вас есть тестовый файл password.txt который содержит набор сгенерированных паролей.  
Ваша задача — проанализировать этот список, чтобы выявить основные статистические характеристики этих паролей.

In [47]:
from pyspark import SparkContext, SparkConf
conf = (SparkConf()
        .setAppName("RDD_analyze_passwords")
        .setMaster("local[*]")
)
sc = SparkContext(conf=conf)

# Загрузка файла в RDD
rdd_passwords = sc.textFile("password.txt")

# Очистка каждой строки от лишних пробелов и удаление пустых строк, если они есть в файле.
rdd_passwords = rdd_passwords.map(lambda line: line.strip())
print(f"Кол-во записей в сыром файле - {rdd_passwords.count()}")
rdd_passwords = rdd_passwords.filter(lambda line: len(line) != 0)
print(f"Кол-во записей после обработки пустых строк - {rdd_passwords.count()}")

# 2. Длина паролей:
# 2.1. Рассчитайте среднюю длину пароля во всем списке. Округлите до целого числа.
# 2.2. Найдите минимальную и максимальную длину паролей.
# 2.3. Определите Топ-5 самых распространенных длин паролей (например, сколько паролей имеют длину 8 символов, сколько — 6 и т.д.).
len_pswd = rdd_passwords.map(lambda line: len(line))
total_sum_pswd = len_pswd.sum()
cnt_strings = rdd_passwords.count()
avg_len_pswd = round(total_sum_pswd / cnt_strings)
print(f"Средняя длина пароля во всем списке - {avg_len_pswd}")

print(f"Минимальная длимна пароля - {len_pswd.min()}")
print(f"Максимальная длимна пароля - {len_pswd.max()}")

tuple_cnt = len_pswd.map(lambda line: (line, 1))
grp_len_passwd = tuple_cnt.reduceByKey(lambda a, b: a + b)
print(f"Топ 5 самых распространенных длин паролей")
print(grp_len_passwd.top(5,key=lambda x: x[1]))

# 3. Символьный состав:
# 3.1. Посчитайте количество паролей, содержащих только цифры.
# 3.2. Посчитайте количество паролей, содержащих только буквы (как строчные, так и заглавные).
digits_rdd = rdd_passwords.filter(lambda line: line.isdigit())
print(f"Кол-во паролей, содержащих только цифры: {digits_rdd.count()}")

alpha_rdd = rdd_passwords.filter(lambda line: line.isalpha())
print(f"Кол-во паролей, содержащих только буквы: {alpha_rdd.count()}")
print()

# 4. Распространенные префиксы/суффиксы:
# 4.1 Определите Топ-5 самых распространенных префикса длиной 3 символа (например, 123, pas).
# 4.2 Определите Топ-5 самых распространенных суффикса длиной 3 символа (например, 678, ord).
lst_prefix = rdd_passwords.map(lambda line: (line[0:3], 1))
cnt_prefix = lst_prefix.reduceByKey(lambda a, b: a + b)
top_5_prefix = cnt_prefix.top(5, key = lambda x: x[1])
print(f"Топ 5 самых распространеных префиксов длиной 3 символа")
print(top_5_prefix)

lst_postfix = rdd_passwords.map(lambda line: (line[-3:], 1))
cnt_postfix = lst_postfix.reduceByKey(lambda a, b: a + b)
top_5_postfix = cnt_postfix.top(5, key = lambda x: x[1])
print(f"Топ 5 самых распространеных постфиксов длиной 3 символа")
print(top_5_postfix)

sc.stop()

Кол-во записей в сыром файле - 1000000
Кол-во записей после обработки пустых строк - 1000000
Средняя длина пароля во всем списке - 9
Минимальная длимна пароля - 6
Максимальная длимна пароля - 15
Топ 5 самых распространенных длин паролей
[(6, 215887), (8, 122106), (7, 115450), (10, 113429), (9, 110343)]
Кол-во паролей, содержащих только цифры: 441347
Кол-во паролей, содержащих только буквы: 115088

Топ 5 самых распространеных префиксов длиной 3 символа
[('123', 17857), ('qwe', 17054), ('pas', 13290), ('654', 9925), ('edc', 9813)]
Топ 5 самых распространеных постфиксов длиной 3 символа
[('123', 16135), ('789', 15266), ('020', 14636), ('021', 14558), ('022', 14552)]


## Решение DataFrame

In [124]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, sum, min, max, length, avg, round, count, substring
from pyspark.sql.types import StructType, StructField, StringType

In [97]:
# Загрузка файла с паролями
spark = SparkSession.builder \
        .appName("RDD_analyze_passwords") \
        .master("local[*]") \
        .config("spark.sql.shuffle.partitions", 4) \
        .getOrCreate()

file_path = "password.txt"
file_schema = StructType([
    StructField("password", StringType(), True)
])

#df_passwords = spark.read.csv(
    #file_path,
    #header=False,
    #schema=file_schema
#)

# грузить напрямую не стоит - надо обрабатывать спецсимволы
df_passwords = spark.read \
        .format("csv") \
        .option("header", "false") \
        .option("inferSchema", "false") \
        .option("wholeFile", "true") \
        .option("encoding", "UTF-8") \
        .option("escape", "") \
        .option("quote", "") \
        .option("sep", "\u0000") \
        .load("password.txt") \
        .withColumnRenamed("_c0", "password")





print(f"Кол-во записей в файле: {df_passwords.count()}")
print(f"Кол-во записей 1-ной длины - {df_passwords.filter(length(col("password")) == 1).count()}")

Кол-во записей в файле: 1000000
Кол-во записей 1-ной длины - 0


In [98]:
# Очистка каждой строки от лишних пробелов и удаление пустых строк, если они есть в файле.
df_cleaned_passwords = df_passwords.withColumn(
    "password",
    trim(col("password"))
)
print(f"Кол-во записей после очистки: {df_cleaned_passwords.count()}")

Кол-во записей после очистки: 1000000


In [118]:
# 2. Длина паролей:
# 2.1. Рассчитайте среднюю длину пароля во всем списке. Округлите до целого числа.
# 2.2. Найдите минимальную и максимальную длину паролей.
# 2.3. Определите Топ-5 самых распространенных длин паролей (например, сколько паролей имеют длину 8 символов, сколько — 6 и т.д.).

df_length_pass = df_cleaned_passwords.withColumn(
    "length_pass",
    length("password")
).orderBy(col("length_pass").desc())

avg_len_pass = df_length_pass.agg(
    round(avg(col("length_pass"))).alias("avg_length_password")
)
avg_len_pass.show()

min_len_pass = df_length_pass.agg(
    min(col("length_pass")).alias("min_length_pass")
)
min_len_pass.show()

max_len_pass = df_length_pass.agg(
    max(col("length_pass")).alias("max_length_pass")
)
max_len_pass.show()

df_top5_length = df_length_pass.groupBy(col("length_pass")).agg(
    count("*").alias("cnt_len")
).orderBy(col("cnt_len").desc())
print(f"Топ 5 распространенных длин паролей")
df_top5_length.show(5)


+-------------------+
|avg_length_password|
+-------------------+
|                9.0|
+-------------------+

+---------------+
|min_length_pass|
+---------------+
|              6|
+---------------+

+---------------+
|max_length_pass|
+---------------+
|             15|
+---------------+

Топ 5 распространенных длин паролей
+-----------+-------+
|length_pass|cnt_len|
+-----------+-------+
|          6| 215887|
|          8| 122106|
|          7| 115450|
|         10| 113429|
|          9| 110343|
+-----------+-------+
only showing top 5 rows



In [123]:
# 3. Символьный состав:
# 3.1. Посчитайте количество паролей, содержащих только цифры.
# 3.2. Посчитайте количество паролей, содержащих только буквы (как строчные, так и заглавные).
df_only_digits = df_cleaned_passwords.filter(col("password").rlike("^\\d+$"))
print(f"Количество паролей, содержащих только цифры - {df_only_digits.count()}")

df_only_letters = df_cleaned_passwords.filter(col("password").rlike("^[A-Za-z]+$"))
print(f"Количество паролей, содержащих только цифры - {df_only_letters.count()}")

Количество паролей, содержащих только цифры - 441347
Количество паролей, содержащих только цифры - 115088


In [131]:
# 4. Распространенные префиксы/суффиксы:
# 4.1 Определите Топ-5 самых распространенных префикса длиной 3 символа (например, 123, pas).
# 4.2 Определите Топ-5 самых распространенных суффикса длиной 3 символа (например, 678, ord).
df_top_prefix = df_cleaned_passwords.withColumn(
    "prefix",
    substring(col("password"), 1, 3)
).groupBy(col("prefix")).agg(
    count("*").alias("cnt_prefix")
).orderBy(col("cnt_prefix").desc())
print(f"Топ-5 самых распространенных префикса длиной 3 символа")
df_top_prefix.show(5)

df_top_postfix = df_cleaned_passwords.withColumn(
    "postfix",
    substring(col("password"), -3, 3)
).groupBy(col("postfix")).agg(
    count("*").alias("cnt_postfix")
).orderBy(col("cnt_postfix").desc())
print(f"Топ-5 самых распространенных суффикса длиной 3 символа")
df_top_postfix.show(5)
sc.stop()

Топ-5 самых распространенных префикса длиной 3 символа
+------+----------+
|prefix|cnt_prefix|
+------+----------+
|   123|     17857|
|   qwe|     17054|
|   pas|     13290|
|   654|      9925|
|   edc|      9813|
+------+----------+
only showing top 5 rows

Топ-5 самых распространенных суффикса длиной 3 символа
+-------+-----------+
|postfix|cnt_postfix|
+-------+-----------+
|    123|      16135|
|    789|      15266|
|    020|      14636|
|    021|      14558|
|    022|      14552|
+-------+-----------+
only showing top 5 rows

